In [2]:
import os
import numpy as np 
from sklearn.model_selection import KFold  # 라벨 x_data, y_data 분포를 유지 (함께 fold)
import tensorflow as tf 
from IPython.display import clear_output

np.random.seed(7)
np.set_printoptions(precision=4, suppress=True)  # 과학적 표기 대신 소숫점 자리 4자리까지 표현

rootDir = "/DockerProjects/Dataset/TUG/trainSet"
expertFolder = "/" + "2021_01_24_saveResults_최윤정"  # --- 변경 할 부분
viewFolder = "/" + "0_sideView"

print(" [Home Dir.] Current path", os.getcwd())  

datasetDir = rootDir + expertFolder + viewFolder
os.chdir(datasetDir) ; print(" Current workaing path [dataset] ", os.getcwd())  # -- Dataset 있는 곳으로 경로 변경 

 [Home Dir.] Current path /DockerProjects/walkCAM/tug/0_temporalCNN_wLabel
 Current workaing path [dataset]  /DockerProjects/Dataset/TUG/trainSet/2021_01_24_saveResults_최윤정/0_sideView


In [3]:
def get_subjectList(datasetDir, shuffle = False):      
    expDates = next(os.walk(datasetDir))[1]   #['2020_11_03', '2020_11_20_v1', '2020_11_20_v2', '2020_11_23', '2020_12_09', '2020_12_30']
    subject_number = 0
    subjects_list = []
    
    for dateFolder in expDates:
        dateDir = os.path.join(datasetDir, dateFolder)   # D:/바탕화면/Dataset/TUG/trainSet/2021_01_24_saveResults_최윤정/0_sideView\2020_11_03
        tmpSubjects = next(os.walk(dateDir))[1]
        subjects_list.append(tmpSubjects)  
        subject_number += len(tmpSubjects)

    # ------ 2D -> 1D [[sub1, sub2], [sub3, sub4]] -> [sub1, sub2, sub3, sub4]
    subjects = []
    for eachSub in subjects_list:
        subjects += eachSub

    if shuffle:
        np.random.shuffle(subjects)
        
    print(" 1D sub list: " , subjects)
    return subjects, subject_number

def fold_trainNames(subList, Kfold_num = 5):
    assert (Kfold_num is not 0), print("check kfold_num. should not be zero")
    Fold_results =[]
    kfold = KFold(n_splits=  Kfold_num)
    
    for trainIdx, testIdx in kfold.split(subList):
        Fold_results.append([trainIdx, testIdx])      
        
    train_Fold= []
    test_Fold = []
        
    # bind fold pair
    for i in range(Kfold_num):
        tmp_train = []
        tmp_test = []
        for subIdx in range(len(subList)): 
            if subIdx in Fold_results[i][0]:  # 
                tmp_train.append(subList[subIdx])
                    
            if subIdx in Fold_results[i][1]:  # -- K-fold test
                tmp_test.append(subList[subIdx])
          
        train_Fold.append(tmp_train)
        test_Fold.append(tmp_test)
        print("\t [process]", i, "-fold:", train_Fold[i], "\n\t\t\t   ,", test_Fold[i])
    print("\n [results]  trainSet: ", len(train_Fold[0]) , " 명 , testSet: ", len(test_Fold[0]), " 명" )
    
    return train_Fold, test_Fold
        

### Train/Test Set -- subject 기준으로 분할   
##### (Kfold_num = 0 이면 직접 분할, 5 등 숫자면 5-fold dataset)

Key    
    https://github.com/ashishpatel26/tcn-keras-Examples  

sub  
      https://www.programmersought.com/article/13674618779/  

In [4]:
train_ratio = 0.6
valid_ratio = 0.2

arbitrarySplit = True
Kfold = False
Kfold_num = 5 # if kfold_num = 0 (arbitrary train/test set will be used)s


numFeature = 3
numActions = 5
dataset_columns = numFeature + numActions

subList, numOfsubjects = get_subjectList(datasetDir, shuffle = False) ; print("\n")

if arbitrarySplit is not True:
    if Kfold:
        print(" K-fold : ", Kfold_num)
        trainFold, testFold = fold_trainNames(subList = subList, Kfold_num = Kfold_num)

        selected_foldNum = 0 

        train_subjects = trainFold[selected_foldNum]
        test_subjects = testFold[selected_foldNum]
        print("\t ------>  Selected Fold num: %d \n   Train Subjects: %s\n   Test Subjects: %s"%(selected_foldNum, train_subjects, test_subjects))
    else:

        trainSub_number = np.round( (numOfsubjects*train_ratio), 0).astype(int)
        validSub_number = np.round( (numOfsubjects*valid_ratio), 0).astype(int)
        testSub_number = numOfsubjects - trainSub_number - validSub_number

        train_subjects = subList[ :trainSub_number]
        valid_subjects = subList[ trainSub_number : (trainSub_number+validSub_number)]
        test_subjects = subList[ (trainSub_number+validSub_number): ]
        print("   No-fold \t [results] \n\t Train: " ,train_subjects, "\n\t Valid: ", valid_subjects, "\n\t Test: ", test_subjects)
        print("\n\t--> # of train  : " + str(trainSub_number) +" 명 ,  # of valid : "+ str(validSub_number) + " 명 ,  # of test :" + str(testSub_number) + "명")

else:
    #         normal_subjects = ['bys_tug', 'cbd_tug', 'cyj_tug', 'kw', 'kyh', 'lhs', 'NHJ_60', 'nhs', 'pjh', 'pss', 'rjh', 'yjh', 'bys', 'cbd', 'cyj', 'cyj2', 'jdh', 'jek', 'kch', 'ryu'] 
#         patients_subjects = ['JDW_tug']
#         copy_patients_subjects = ['BYS_TUG', 'CYJ_TUG', 'PHE_TUG', 'YSJ_TUG'] 

    train_subjects = ['bys_tug', 'cbd_tug', 'cyj_tug', 'kw', 'kyh', 'nhs', 'pjh', 'pss', 'rjh', 'yjh', 'bys', 'cbd', 'cyj', ] 
    valid_subjects = ['cyj2', 'jdh', 'jek', 'lhs' ] 
    test_subjects =  ['kch', 'ryu', 'NHJ_60']
    print(" No-fold --- (arbitrary) \t [results] \n\t Train: " ,train_subjects, "\n\t Valid: ", valid_subjects, "\n\t Test: ", test_subjects)

 1D sub list:  ['bys_tug', 'cbd_tug', 'cyj_tug', 'pss', 'yjh', 'lhs', 'kyh', 'nhs', 'kw', 'NHJ_60', 'pjh', 'rjh', 'ryu', 'JDW_tug', 'PHE_TUG', 'YSJ_TUG', 'CYJ_TUG', 'BYS_TUG', 'jdh', 'cbd', 'kch', 'bys', 'jek', 'cyj', 'cyj2']


 No-fold --- (arbitrary) 	 [results] 
	 Train:  ['bys_tug', 'cbd_tug', 'cyj_tug', 'kw', 'kyh', 'nhs', 'pjh', 'pss', 'rjh', 'yjh', 'bys', 'cbd', 'cyj'] 
	 Valid:  ['cyj2', 'jdh', 'jek', 'lhs'] 
	 Test:  ['kch', 'ryu', 'NHJ_60']


In [5]:
def getPelvisData(csvFile):
    rawData = np.loadtxt(csvFile, delimiter=",")
    timestamp = rawData[:,0]
    pelvis_x = rawData[:,1]
    pelvis_y = rawData[:,2]
    pelvis_z = rawData[:,3]
    actionList = rawData[:,4:9]
    return timestamp, pelvis_x, pelvis_y, pelvis_z, actionList

def sliding_window(timestamp, data_x, data_y, lookback_window=3):
    time = []
    x = []
    y = []
    enc = MinMaxScaler(feature_range = (0, 1))
    enc_y = enc.fit_transform(data_y)
    for i in range(lookback_window, len(data_x)+1):
        time.append(timestamp[i-1])
        x.append(data_x[i - lookback_window:i])
        y.append(data_y[i-1])
  
    x = np.array(x)
    x = x.reshape(-1, lookback_window, numFeature, 1)
    
    y = np.array(y)
    time = np.array(time)
   
    return time, x, y, enc


def create_dataset(train_subjects, test_subjects, lookback_window = 3):
    trainSet = None
    validSet = None
    testSet = None
    
    for dirpath, foldername, files in sorted(os.walk(datasetDir)):
        for filename in files:
            if "lpf_" in filename:
                subname = dirpath.split("/")[-2] 
                csvFile = os.path.join(dirpath, filename)
                          
                timestamp, pelvis_x, pelvis_y, pelvis_z, actionList = getPelvisData(csvFile) 
                
                pelvisData = np.array([pelvis_x, pelvis_y, pelvis_z]).T  # (317,3)
                actionData = np.array(actionList)    # # (335, 5)
    
                blockTime, blockPelvis, blockLable, enc = sliding_window(timestamp, data_x = pelvisData, data_y = actionData, lookback_window=lookback_window) # pelvis - (327, 8, 3)
                # bind as a tensor
                dataset = tf.data.Dataset.from_tensor_slices( (blockPelvis.astype('float32'), blockLable.astype('float32')) )
                
                if subname in train_subjects:
                    if trainSet is None:
                        trainSet = dataset
                    else:
                        trainSet = trainSet.concatenate(dataset)
                        
                elif subname in test_subjects:
                    if testSet is None:
                        testSet = dataset
                    else:
                        testSet = testSet.concatenate(dataset)
                else:
                    if validSet is None:
                        validSet = dataset
                    else:
                        validSet = validSet.concatenate(dataset)       
            
    return trainSet, validSet, testSet


### Test section

In [6]:
def show_batch(dataset):
    for batch, label in dataset.take(1):
        print("{:20s}: {}".format('Label',label.numpy()))
        for key, value in batch.items():
            print("{:20s}: {}".format(key,value.numpy()))
import tensorflow as tf
import numpy as np

data_x = np.random.rand(8)
data_y = np.array(range(8))

print(data_x,"\n", data_y,"\n")
x = []
y = []
lookback_window = 3
for i in range(lookback_window, len(data_x)+1):
    x.append(data_x[i - lookback_window:i])    
    y.append(data_y[i-1])
x = np.array(x)
y = np.array(y)


dataset = tf.data.Dataset.from_tensor_slices( (x.astype('float32'), y.astype('float32')) )

# for element in dataset:
#     print(element)
print(dataset.element_spec)

[0.0763 0.7799 0.4384 0.7235 0.978  0.5385 0.5011 0.0721] 
 [0 1 2 3 4 5 6 7] 

(TensorSpec(shape=(3,), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.float32, name=None))


In [7]:
from sklearn.preprocessing import MinMaxScaler

lookback_window = 8
batch_size = 256
epochs = 150
learning_rate = 0.00001
trainSet, validSet, testSet = create_dataset(train_subjects, test_subjects, lookback_window = lookback_window)
trainSet_shuffled = trainSet.shuffle(10000).batch(batch_size, drop_remainder = True)

print(trainSet.element_spec)
print(trainSet_shuffled.element_spec)

print(len(trainSet_shuffled))
# cnt =0
# print(len(trainSet))
# print(len(trainSet_shuffled))
# for element in trainSet:
#     print(element[0], element[1])
#     cnt+=1
#     if cnt == 5:
#         break

(TensorSpec(shape=(8, 3, 1), dtype=tf.float32, name=None), TensorSpec(shape=(5,), dtype=tf.float32, name=None))
(TensorSpec(shape=(256, 8, 3, 1), dtype=tf.float32, name=None), TensorSpec(shape=(256, 5), dtype=tf.float32, name=None))
185


### Model Build

In [8]:
from keras.layers import Conv1D, Input, Activation, Flatten, Dense, Conv2D
from keras.layers import BatchNormalization, add , Dropout
from keras.models import Model
from keras.optimizers import Adam

from keras.callbacks import ModelCheckpoint, EarlyStopping

from tcn import TCN

In [9]:
MODEL_SAVE_FOLDER_PATH = rootDir + "/"+ expertFolder +"/model_TCN/"

print(MODEL_SAVE_FOLDER_PATH)
if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
    os.mkdir(MODEL_SAVE_FOLDER_PATH)
    
model_path = MODEL_SAVE_FOLDER_PATH + "WinSize- "+ str(lookback_window) +"TUG-" + "{epoch:02d} ---- {val_loss: .4f}.hdf5"

cb_checkpoint = ModelCheckpoint(filepath=model_path, monitor="val_loss", verbose=1, save_best_only=True)
cb_early_stopping = EarlyStopping(monitor="val_loss", patience=15)

/DockerProjects/Dataset/TUG/trainSet//2021_01_24_saveResults_최윤정/model_TCN/


In [1]:
def custom_loss(y_actual,y_pred):
    y_ = kb.sigmoid(y_pred)
    custom_loss = constant * y_actual * kb.log(y_) + (1- y_actual) * kb.log(1 - y_)
    return -tf.reduce_mean(custom_loss)

class RNN(Model):

    def __init__(self):
        super(RNN, self).__init__()
        self.lstm1 = LSTM(15, return_sequences=True)
        self.lstm2 = LSTM(10, return_sequences=True)
        self.lastLayer = TimeDistributed(Dense(1, activation = 'sigmoid'))

    def call(self, inputs):
        x = self.lstm1(inputs)
        x = self.lstm2(x)
        x = self.lastLayer(x)
        print('rnn', x.shape)
        return x
    
    
    
 #Residual block :: https://roadcom.tistory.com/95
def ResBlock(x,filters,kernel_size,dilation_rate):
    r=Conv2D(filters,kernel_size=kernel_size,padding='same',dilation_rate=dilation_rate,activation='relu')(x) #first convolution
    r=Conv2D(filters,kernel_size=kernel_size,padding='same',dilation_rate=dilation_rate)(r) #Second convolution
    if x.shape[-1]==filters:
        # Shortcut 의 channel 과 main path 의 channel 이 일치할 경우 단순 add 연산만 진행하는 블록 = identity block
        shortcut = x  # identity block 
    else: 
        # Shortcut 의 channel 과 main path 의 channel 이 다를 경우 shortcut path 를 적절히 변환
        # 즉, projection 을 통해 channel 을 맞춰주는 작업이(projection shortcut) 추가되기에 이를 convolution block 이라함
        shortcut=Conv2D(filters,kernel_size=kernel_size,padding='same')(x) 
    o=add([r,shortcut])
    o=Activation('relu')(o) 
    return o
 
 #Sequence Model
def TCN(optimizer, class_weight):
    kernel_size = (3,3)
    input_shape =  (lookback_window, 3, 1) # (3, 8, 1) = (feature, sliding_window, 1)
    
    inputs=Input(shape=input_shape)
    
    x=ResBlock(x = inputs,filters=32,kernel_size=kernel_size,dilation_rate=1)
    x = Dropout(0.2) (x)
    x=ResBlock(x,filters=32,kernel_size=kernel_size,dilation_rate=2)
    x = Dropout(0.4)(x)
    x=ResBlock(x,filters=16,kernel_size=kernel_size,dilation_rate=4)
    
    x=Flatten()(x)
    x=Dense(5, activation='softmax')(x)
    model=Model(inputs=inputs,outputs=x)
         
    model.summary()
    model.compile(optimizer=optimizer,loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

NameError: name 'Model' is not defined

In [1]:
    
optimizer = tf.optimizers.Adam(learning_rate)
# (feature, sliding_window, 1) ...TCN,,,  
class_weight = {0:3.298, 1:1.0, 2:6.929, 3:2.052, 4:1.427}
model = TCN(optimizer, class_weight)
# trainSet  trainSet_shuffled
# hist = model.fit(trainSet_shuffled, batch_size= batch_size, epochs=epochs,verbose=2, validation_data= validSet,  callbacks=[cb_checkpoint, cb_early_stopping], class_weight=class_weight)
# eval_result = model.evaluate(trainSet_shuffled,batch_size= batch_size,verbose=2)    
hist = model.fit(trainSet, epochs=epochs,verbose=2, validation_data= validSet,  callbacks=[cb_checkpoint, cb_early_stopping], class_weight=class_weight)
model.fit(dataset_shuffled, epochs=epoch)
# eval_result = model.evaluate(testSet,verbose=2)    
 

print('test_loss:',eval_result[0],'- test_acc:',eval_result[1])

NameError: name 'tf' is not defined

In [ ]:
# y_pred = model.predict(x_test)
y_pred = model.predict(x_test)
print(y_pred.shape)

y_pred_onehot = np.argmax(y_pred, axis=1)
print(y_pred[200] ,"===> onehot: ", y_pred_onehot[200])


In [ ]:

import matplotlib.pyplot as plt

fig, loss_ax = plt.subplots()
acc_ax = loss_ax.twinx()

loss_ax.plot(hist.history['loss'], 'y', label='train loss')
loss_ax.plot(hist.history['val_loss'], 'r', label='val loss')
loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
loss_ax.legend(loc='upper left')

acc_ax.plot(hist.history['accuracy'], 'b', label='train acc')
acc_ax.plot(hist.history['val_accuracy'], 'g', label='val acc')
acc_ax.set_ylabel('accuracy')
acc_ax.legend(loc='lower left')

plt.show()


### Confusion Matrix, Precision, Recall, F1 score

In [ ]:
y_test_argmax = np.argmax(y_test ,axis=1) 
y_pred_argmax = np.argmax(y_pred ,axis=1)
print(y_test_argmax.shape, y_pred_argmax.shape)

def action_frames(y_pred_argmax, ratio_display = False):
    action_label =  {"sit": 0, "sit-stand": 1,  "walk": 2, "turn": 3, "stand-sit": 4}
    action_cnt =  {"total_frames": 0,"sit": 0, "sit-stand": 0,  "walk": 0, "turn": 0, "stand-sit": 0}

    for i in range(len(y_pred_argmax)):
        action_cnt["total_frames"] +=1
        if y_pred_argmax[i] == action_label["sit"]:
            action_cnt["sit"] +=1
            
        elif y_pred_argmax[i] == action_label["sit-stand"]:
            action_cnt["sit-stand"] +=1
            
        elif y_pred_argmax[i] == action_label["walk"]:
            action_cnt["walk"] +=1
            
        elif y_pred_argmax[i] == action_label["turn"]:
            action_cnt["turn"] +=1
            
        elif y_pred_argmax[i] == action_label["stand-sit"]:
            action_cnt["stand-sit"] +=1
        
    if ratio_display:     
        print("[ Action Ratio ]")
        print("\t [ 0 - Sit] ratio of sit: ",  action_cnt["sit"]/action_cnt["total_frames"] )
        print("\t [ 1 - sit-stand] ratio of sit-stand: ",  action_cnt["sit-stand"]/action_cnt["total_frames"])
        print("\t [ 2 - walk] ratio of walk: ",  action_cnt["walk"]/action_cnt["total_frames"])
        print("\t [ 3 - turn] ratio of turn: ",  action_cnt["turn"]/action_cnt["total_frames"])
        print("\t [ 4 - stand-sit] ratio of stand-sit: ",  action_cnt["stand-sit"]/action_cnt["total_frames"])


    
    return action_cnt
    
y_pred_frames = action_frames(y_pred_argmax, ratio_display= True)
y_test_frames = action_frames(y_test_argmax,  ratio_display= True)   

print("--->  # of Each Action Frames  \n\t y_pred: {0}, \n\t y_test: {1}".format(y_pred_frames, y_test_frames))

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import cohen_kappa_score
import seaborn as sns


kappa = cohen_kappa_score(y_test_argmax, y_pred_argmax)
print('Cohens kappa: %f' % kappa)

print(classification_report(y_test_argmax, y_pred_argmax))

conf_matrix = confusion_matrix(y_test_argmax, y_pred_argmax)
print("\n---> Confusion Matrix \n" ,conf_matrix) # sit, sit-stand, walking, turning, stand-sit

plt.figure(figsize = (15,7) )
sns.heatmap(conf_matrix, annot=True)
plt.title("Confusion Matrix")
plt.show()

import matplotlib.pyplot as plt

print("Testing Accuracy: {}%".format(100*accuracy))

print("")
print("Precision: {}%".format(100*metrics.precision_score(y_test, predictions, average="weighted")))
print("Recall: {}%".format(100*metrics.recall_score(y_test, predictions, average="weighted")))
print("f1_score: {}%".format(100*metrics.f1_score(y_test, predictions, average="weighted")))

print("")
print("Confusion Matrix:")
confusion_matrix = metrics.confusion_matrix(y_test, predictions)
print(confusion_matrix)
normalised_confusion_matrix = np.array(confusion_matrix, dtype=np.float32)/np.sum(confusion_matrix)*100

print("")
print("Confusion matrix (normalised to % of total test data):")
print(normalised_confusion_matrix)
print("Note: training and testing data is not equally distributed amongst classes, ")
print("so it is normal that more than a 6th of the data is correctly classifier in the last category.")

# Plot Results: 
width = 12
height = 12
plt.figure(figsize=(width, height))
plt.imshow(
    normalised_confusion_matrix, 
    interpolation='nearest', 
    cmap=plt.cm.rainbow
)
plt.title("Confusion matrix \n(normalised to % of total test data)")
plt.colorbar()
tick_marks = np.arange(n_classes)
plt.xticks(tick_marks, LABELS, rotation=90)
plt.yticks(tick_marks, LABELS)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()